## 한국어 데이터로 챗봇 만들기
### Trnsformer_Chatbot
***
#### Step 1. 데이터 수집하기
-송영숙님의 챗봇 데이터를 사용하였습니다. 

#### Step 2. 데이터 전처리하기
-영어 데이터와는 전혀 다른 데이터인 만큼 영어 데이터에 사용했던 전처리와  
일부 동일한 전처리도 필요하겠지만 전체적으로는 다른 전처리를 수행해야 할 수도 있습니다. 

#### Step 3. SubwordTextEncoder 사용하기
-한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다.  
-하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 내부 단어 토크나이저인  
SubwordTextEncoder를 그대로 사용해보세요.

#### Step 4. 모델 구성하기
-위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다. 

#### Step 5. 모델 평가하기
-Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.

***
#### 평가문항
**1. 한국어 전처리를 통해 학습 데이터셋을 구축하였다.**  
-공백과 특수문자 처리, 토크나이징, 병렬데이터 구축의 과정이 적절히 진행되었다.

**2. 트랜스포머 모델을 구현하여 한국어 챗봇 모델 학습을 정상적으로 진행하였다.**  
-구현한 트랜스포머 모델이 한국어 병렬 데이터 학습 시 안정적으로 수렴하였다.

**3. 한국어 입력문장에 대해 한국어로 답변하는 함수를 구현하였다.**  
-한국어 입력문장에 그럴듯한 한국어로 답변을 리턴하였다.
***
#### Step 1. 데이터 수집하기
-송영숙님의 챗봇 데이터를 사용하였습니다. 
***

In [32]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# 데이터 가져오기
chat_filepath = os.getenv('HOME') + '/aiffel/songys_chatbot/Chatbot_data/ChatbotData .csv'
chat_data = pd.read_csv(chat_filepath, sep=',')
chat_data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
# 전처리 함수
def preprocess_sentence(sentence):
    #한글은 대소문자 구분이 필요없으므로 해당 코드 주석처리
    #sentence = sentence.lower().strip()

    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    # 0박0일 같은 부분은 숫자가 생략되면 안되기때문에 영어와 더불어 한글, 
    # 숫자도 생략하지 않도록 합니다.
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ][a-zA-Z][0-9]", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [4]:
clean_Q=[]
for s in chat_data['Q']:
    clean_Q.append(preprocess_sentence(s))
clean_Q[:5]

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']

In [5]:
clean_A=[]
for s in chat_data['A']:
    clean_A.append(preprocess_sentence(s))
clean_A[:5]

['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']

In [6]:
print('전체 샘플 수(질문) :', len(clean_Q))
print('전체 샘플 수(답변) :', len(clean_A))
print('전처리 후의 50번째 질문 샘플: {}'.format(clean_Q[49]))
print('전처리 후의 50번째 답변 샘플: {}'.format(clean_A[49]))

전체 샘플 수(질문) : 11823
전체 샘플 수(답변) : 11823
전처리 후의 50번째 질문 샘플: 감 말랭이 먹고 싶다 .
전처리 후의 50번째 답변 샘플: 맛있게 드세요 .


In [7]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(clean_Q + clean_A, target_vocab_size=2**13)

In [8]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8173]
END_TOKEN의 번호 : [8174]


In [9]:
VOCAB_SIZE = tokenizer.vocab_size + 2

In [10]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 50번째 질문 샘플: {}'.format(tokenizer.encode(clean_Q[50])))
print('정수 인코딩 후의 50번째 답변 샘플: {}'.format(tokenizer.encode(clean_A[50])))

정수 인코딩 후의 50번째 질문 샘플: [3173, 307, 8152, 8075, 8090, 11, 670]
정수 인코딩 후의 50번째 답변 샘플: [502, 126, 1]


In [11]:
Q_len = [len(s.split()) for s in chat_data['Q']]
A_len = [len(s.split()) for s in chat_data['A']]

print('Q의 최소 길이 : {}'.format(np.min(Q_len)))
print('Q의 최대 길이 : {}'.format(np.max(Q_len)))
print('Q의 평균 길이 : {}'.format(np.mean(Q_len)))
print('Q의 표준 편차 : {}'.format(np.std(Q_len)))
print('A의 최소 길이 : {}'.format(np.min(A_len)))
print('A의 최대 길이 : {}'.format(np.max(A_len)))
print('A의 평균 길이 : {}'.format(np.mean(A_len)))
print('A의 표준 편차 : {}'.format(np.std(A_len)))

Q의 최소 길이 : 1
Q의 최대 길이 : 15
Q의 평균 길이 : 3.587414361837097
Q의 표준 편차 : 1.6188281321513895
A의 최소 길이 : 1
A의 최대 길이 : 21
A의 평균 길이 : 3.6936479742874058
A의 표준 편차 : 1.8572136133798673


In [12]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s.split()) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))))

In [13]:
Q_max_len = int(np.mean(Q_len)+2*np.std(Q_len))
A_max_len = int(np.mean(A_len)+2*np.std(A_len))
below_threshold_len(Q_max_len, chat_data['Q'])
below_threshold_len(A_max_len, chat_data['A'])
max_length = 9

전체 샘플 중 길이가 6 이하인 샘플의 비율: 0.9461219656601539
전체 샘플 중 길이가 7 이하인 샘플의 비율: 0.9622769178719445


In [14]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs,MAX_LENGTH):
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
        # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
  
    # 최대 길이 40으로 모든 데이터셋을 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs

In [15]:
clean_Q, clean_A = tokenize_and_filter(clean_Q, clean_A,9)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(clean_Q)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(clean_A)))

단어장의 크기 : 8175
필터링 후의 질문 샘플 개수: 7806
필터링 후의 답변 샘플 개수: 7806


In [25]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': clean_Q,
        'dec_inputs': clean_A[:, :-1]
    },
    {
        'outputs': clean_A[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        # 배열의 짝수 인덱스에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])
        # 배열의 홀수 인덱스에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
    

In [28]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    # 인코더에서 패딩을 위한 마스크
    enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, 
                                              output_shape=(1, 1, None),
                                              name='enc_padding_mask')(inputs)

    # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
    # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask,
                                             output_shape=(1, None, None),
                                             name='look_ahead_mask')(dec_inputs)

    # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
    # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask, 
                                              output_shape=(1, 1, None),
                                              name='dec_padding_mask')(inputs)

    # 인코더
    enc_outputs = encoder(vocab_size=vocab_size,
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                         )(inputs=[inputs, enc_padding_mask])

    # 디코더
    dec_outputs = decoder(vocab_size=vocab_size,
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                         )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [40]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수
D_MODEL = 512 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(vocab_size=VOCAB_SIZE,
                    num_layers=NUM_LAYERS,
                    units=UNITS,
                    d_model=D_MODEL,
                    num_heads=NUM_HEADS,
                    dropout=DROPOUT)

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


TypeError: Input 'y' of 'Pow' Op has type float32 that does not match type int32 of argument 'x'.

In [17]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query,key,value,mask):
    # 어텐션 가중치를 계산
    matmul_qk=tf.matmul(query,key,transpose_b=True)
    
    # scale matmul_qk
    depth=tf.cast(tf.shape(key)[-1],tf.float32)
    logits=matmul_qk/tf.math.sqrt(depth)
    
    # add the mask to zero outpadding tokens
    if mask is not None:
        logits+=(mask*-1e9)
        
    # softmax is normalized on the last axis(seq_len_k)
    attention_weights=tf.nn.softmax(logits,axis=-1)
    
    output=tf.matmul(attention_weights,value)
    
    return output

In [18]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads=num_heads
        self.d_model=d_model
        
        assert d_model%self.num_heads==0
        
        self.depth=d_model//self.num_heads
        
        self.query_dense=tf.keras.layers.Dense(units=d_model)
        self.key_dense=tf.keras.layers.Dense(units=d_model)
        self.value_dense=tf.keras.layers.Dense(units=d_model)
        
        self.dense=tf.keras.layers.Dense(units=d_model)
        
    def split_heads(self, inputs, batch_size):
        inputs=tf.reshape(inputs,
                         shape=(batch_size,-1,self.num_heads,self.depth))
        return tf.transpose(input,perm=[0,2,1,3])
    
    def call(self,inputs):
        query,key,value,mask=inputs['query'],inputs['key'],inputs['value'],inputs['mask']
        batch_size=tf.shape(query)[0]
        
        # linear layers
        query=self.query_dense(query)
        key=self.key_dense(key)
        value=self.value_dense(value)
        
        # 병렬 연산을 위해 머리를 여러개 만듭니다.
        query=self.split_heads(query,batch_size)
        key=self.split_heads(key,batch_size)
        value=self.split_heads(value,batch_size)
        
        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention=scaled_dot_product_attention(query,key,value,mask)
        
        scaled_attention=tf.transpose(scaled_attention, perm[0,2,1,3])
        
        #어텐션 연산 후에 각 결과를 다시 연결하기
        concat_attention=tr.reshape(scaled_attenrion,
                                   batch_size,-1,
                                   self.d_model)
        
        #final linear layer
        output=self.dense(concat_attenrion)


In [19]:
def create_padding_mask(x):
    mask=tf.cast(tf.math.equal(x,0),tf.float32)
    # (batch_size,1,1,sequence length)
    return mask[:,tf.newaxis,tf.newaxis,:]

In [36]:
def create_look_ahead_mask(x):
    seq_len=tf.shape(x)[1]
    look_ahead_mask=1-tf.linalg.band_part(tf.ones((seq_len,seq_len)),-1,0)
    padding_mask=create_padding_mask(x)
    return tf.maximum(look_ahead_mask,padding_mask)

In [21]:
# 인코더 하나의 레이어를 함수로 구현합니다.
# 이 하나의 레이어 안에는 두개의 서브 레이어가 존재합니다.
def encoder_layer(units,d_model,num_heads,dropout,name="encoder_layer"):
    inputs=tf.keras.input(shape=(None,d_model),name="inputs")
    
    # 패딩 마스크 사용
    padding_mask=tf.keras.input(shape=(1,1,None),name="padding_mask")
    
    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행(셀프 어텐션)
    attention=MultiHeadAttention(d_model,num_heads,name="attention")({
        'query':inputs,'key':inputs,'value':inputs,'mask':padding_mask
    })
    
    #어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
    attention=tf.keras.layers.Dropout(rate=dropout)(attention)
    attention=tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs+attention)
    
    # 두번째 서브 레이어 : 2개의 완전 연결층
    outputs=tf.keras.layers.Dense(units=units,activation='relu')(attention)
    outputs=tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention+outputs)
    
    return tf.keras.Model(inputs=[inputs,padding_mask],outputs=outputs,name=name)
    

In [39]:
# 인코더 층을 쌓아서 인코더를 만들어보겠습니다.
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    # num_layers만큼 쌓아올린 인코더의 층.
    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [23]:
# 디코더 하나의 레이어를 하나의 함수로 구현합니다.
# 이 하나의 레이어 안에는 세개의 서브 레이어가 존재합니다.
def decoder_layer(units,d_model,num_heads,dropout,name="decoder_layer"):
    inputs=tf.keras.input(shape=(None,d_medel),name="inputs")
    enc_outputs=tf.keras.input(shape=(None,d_model),name="encoder_outputs")
    look_ahead_mask=tf.keras.input(shape=(1,None,None),name="look_ahead_mask")
    padding_mask=tf.keras.input(shape=(1,1,None),name='padding_mask')
    
    # 첫번째 서브 레이어 : 멀티 헤드 어텐션 수행(셀프 어텐션)
    attention1 = MultiHeadAttention(d_model,num_heads,name="attention_1")(inputs={
        'query':inputs,'key':inputs,'value':inputs,'mask':look_ahead_mask
    })
    
    # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1=tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1+inputs)
    
    # 두번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행(인코더-디코더 어텐션)
    attention2=MultiHeadAttention(d_model,num_heads,name="attention_2")(inputs={
        'query':attention1,'key':enc_outputs,'value':enc_outputs,'mask':padding_mask
    })
    
    
    # 마스크드 멀티 헤드 어텐션의 결과는
    # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention2=tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2=tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2+attention1)
    
    # 세번째 서브 레이어 : 2개의 완전연결층
    outputs=tf.keras.layers.Dense(units=units,activation='relu')(attention2)
    outputs=tf.keras.layers.Dense(units=d_model)(outputs)
    
    # 완전연결층의 결과는 Dropout과 LayerNormalizaion을 수행합니다.
    outputs=tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs=tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs+attention2)
    
    return tf.keras.Model(inputs=[inputs,enc_outputs,look_ahead_mask,padding_mask],
                         ouputs=outputs,name=name)

In [24]:
# 디코더 층을 쌓아 디코더 만들기
def decoder(vocab_size, num_layers, units, d_model, num_heads, dropout, name='decoder'):
    inputs=tf.keras.input(shape=(None,),name='inputs')
    enc_outputs=tf.keras.input(shape-(None,d_model),name='encoder_outputs')
    look_ahead_mask=tf.keras.input(shape=(1,None,None),name='look_ahead_mask')
    
    # 패딩 마스크
    padding_mask=tf.keras.input(shape=(1,1,None),name='paddint_mask')
    
    # 임베딩 레이어
    embeddings=tf.keras.layers.Embedding(vocab_size,d_model)(inputs)
    embeddings*=tf.math.sqrt(tf.cast(d_model,tf.float32))
    
    # 포지셔널 인코딩
    embeddings=PositionalEncoding(vocab_size,d_model)(embeddings)
    
    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs=tf.keras.layers.Dropout(rate=dropout)(embeddings)
    
    for i in range(num_layers):
        outputs=decoder_layer(units=units,d_model=d_model,num_heads=num_heads,
                             dropout=dropout,name='decoder_layer_{}'.format(i),)(
        inputs=[outputs,enc_outputs,look_ahead_mask,padding_mask])
        
        return tf.keras.Model(inputs=[inputs,enc_outputs,look_ahead_mask,padding_mask],
                             outputs=outputs, name=name)